In [1]:
#Natural Language Toolkit for Natural Language Processing
!pip install autocorrect

     |████████████████████████████████| 622 kB 4.8 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=7991c0b491efec5e91203a9523381d062e3d88d4db627dcdc13440bc7134cbdd
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect


Import neccasary libraries and import data from github

In [2]:
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import string
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
comment_words = ' '
stopwords = set(STOPWORDS) 
from autocorrect import Speller
from nltk.tokenize import WhitespaceTokenizer
from nltk import word_tokenize
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
def clean_text(text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', text)
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace

#Code for Post_Text

In [4]:
herpes1 = pd.read_csv("https://raw.githubusercontent.com/SrushtiChauhan/Test-2022/main/HERPES_comments_main.csv", 
                          on_bad_lines='skip');

In [5]:
print('HERPES Posts data:', herpes1.shape)

HERPES Posts data: (990, 5)


In [6]:
#Data Cleaning
herpes1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              990 non-null    object
 1   Post_Text       990 non-null    object
 2   Total_Comments  990 non-null    int64 
 3   Score           990 non-null    int64 
 4   Post_Created    990 non-null    object
dtypes: int64(2), object(3)
memory usage: 38.8+ KB


In [7]:
herpes1.describe()

,Total_Comments,Score
count,990.000000,990.000000
mean,8.311111,4.621212
std,15.284567,11.511324
min,0.000000,0.000000
25%,2.000000,1.000000
50%,4.000000,2.000000
75%,9.000000,3.000000
max,184.000000,172.000000


In [8]:
herpes1.isnull().sum()

ID                0
Post_Text         0
Total_Comments    0
Score             0
Post_Created      0
dtype: int64

In [9]:
# drop all rows with any NaN and NaT values
herpes1 = herpes1.dropna()

In [10]:
herpes1.isnull().sum()

ID                0
Post_Text         0
Total_Comments    0
Score             0
Post_Created      0
dtype: int64

In [11]:
# Selecting duplicate rows except first
# occurrence based on all columns
duplicate = herpes1[herpes1.duplicated()]
 
print("Duplicate Rows :")
 
# Print the resultant Dataframe
duplicate

Duplicate Rows :


,ID,Post_Text,Total_Comments,Score,Post_Created


In [12]:
#Data Pre-Processing
herpes1.columns

Index(['ID', 'Post_Text', 'Total_Comments', 'Score', 'Post_Created'], dtype='object')

In [13]:
herpes1.head(3)

,ID,Post_Text,Total_Comments,Score,Post_Created
0,60xlxh,Be sure to read the sub RULES before asking ab...,24,56,2017-03-22 17:33:44-04:00
1,npznfp,SPFPP 2021 HSV Survey Results Are in Here’s th...,142,172,2021-06-01 13:36:45-04:00
2,tynsw3,Hey! You are more than herpes! ✨ I don’t know ...,12,9,2022-04-07 17:50:44-04:00


In [14]:
#Convert comments data into lower case
herpes1['clean_text'] = herpes1["Post_Text"].str.lower()
herpes1

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text
0,60xlxh,Be sure to read the sub RULES before asking ab...,24,56,2017-03-22 17:33:44-04:00,be sure to read the sub rules before asking ab...
1,npznfp,SPFPP 2021 HSV Survey Results Are in Here’s th...,142,172,2021-06-01 13:36:45-04:00,spfpp 2021 hsv survey results are in here’s th...
2,tynsw3,Hey! You are more than herpes! ✨ I don’t know ...,12,9,2022-04-07 17:50:44-04:00,hey! you are more than herpes! ✨ i don’t know ...
3,typwuc,Day 1 I just left the Planned Parenthood where...,1,4,2022-04-07 19:33:30-04:00,day 1 i just left the planned parenthood where...
4,typdhg,Western Blot Update I already knew I had hsv1 ...,4,6,2022-04-07 19:06:32-04:00,western blot update i already knew i had hsv1 ...
...,...,...,...,...,...,...
985,ticgwt,Can blisters go away without bursting?,8,2,2022-03-19 23:47:10-04:00,can blisters go away without bursting?
986,ti8bec,Scared to disclose So it’s been a couple month...,7,4,2022-03-19 19:55:19-04:00,scared to disclose so it’s been a couple month...
987,ti8tus,Hi. I just got diagnosed today and I am devast...,4,3,2022-03-19 20:21:23-04:00,hi. i just got diagnosed today and i am devast...
988,tidj6p,First time disclosing Met someone on tinder an...,3,1,2022-03-20 00:52:55-04:00,first time disclosing met someone on tinder an...


In [15]:
#Remove punctuation and special characters from text data
herpes1["clean_text"] = [clean_text(t) for t in herpes1['clean_text']]
herpes1

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text
0,60xlxh,Be sure to read the sub RULES before asking ab...,24,56,2017-03-22 17:33:44-04:00,be sure to read the sub rules before asking ab...
1,npznfp,SPFPP 2021 HSV Survey Results Are in Here’s th...,142,172,2021-06-01 13:36:45-04:00,spfpp hsv survey results are in here’s the res...
2,tynsw3,Hey! You are more than herpes! ✨ I don’t know ...,12,9,2022-04-07 17:50:44-04:00,hey you are more than herpes ✨ i don’t know if...
3,typwuc,Day 1 I just left the Planned Parenthood where...,1,4,2022-04-07 19:33:30-04:00,day i just left the planned parenthood where t...
4,typdhg,Western Blot Update I already knew I had hsv1 ...,4,6,2022-04-07 19:06:32-04:00,western blot update i already knew i had hsv b...
...,...,...,...,...,...,...
985,ticgwt,Can blisters go away without bursting?,8,2,2022-03-19 23:47:10-04:00,can blisters go away without bursting
986,ti8bec,Scared to disclose So it’s been a couple month...,7,4,2022-03-19 19:55:19-04:00,scared to disclose so it’s been a couple month...
987,ti8tus,Hi. I just got diagnosed today and I am devast...,4,3,2022-03-19 20:21:23-04:00,hi i just got diagnosed today and i am devasta...
988,tidj6p,First time disclosing Met someone on tinder an...,3,1,2022-03-20 00:52:55-04:00,first time disclosing met someone on tinder an...


In [16]:
#Autocorrect and spelling mistake correction
#spell = Speller(lang='en')

#herpes1["clean_text"] = [spell(t) for t in herpes1["clean_text"]]
#herpes1

In [17]:
#Tokenization
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()

herpes1["tokenization"] = [tk.tokenize(t) for t in herpes1["clean_text"]]
herpes1

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text,tokenization
0,60xlxh,Be sure to read the sub RULES before asking ab...,24,56,2017-03-22 17:33:44-04:00,be sure to read the sub rules before asking ab...,"[be, sure, to, read, the, sub, rules, before, ..."
1,npznfp,SPFPP 2021 HSV Survey Results Are in Here’s th...,142,172,2021-06-01 13:36:45-04:00,spfpp hsv survey results are in here’s the res...,"[spfpp, hsv, survey, results, are, in, here’s,..."
2,tynsw3,Hey! You are more than herpes! ✨ I don’t know ...,12,9,2022-04-07 17:50:44-04:00,hey you are more than herpes ✨ i don’t know if...,"[hey, you, are, more, than, herpes, ✨, i, don’..."
3,typwuc,Day 1 I just left the Planned Parenthood where...,1,4,2022-04-07 19:33:30-04:00,day i just left the planned parenthood where t...,"[day, i, just, left, the, planned, parenthood,..."
4,typdhg,Western Blot Update I already knew I had hsv1 ...,4,6,2022-04-07 19:06:32-04:00,western blot update i already knew i had hsv b...,"[western, blot, update, i, already, knew, i, h..."
...,...,...,...,...,...,...,...
985,ticgwt,Can blisters go away without bursting?,8,2,2022-03-19 23:47:10-04:00,can blisters go away without bursting,"[can, blisters, go, away, without, bursting]"
986,ti8bec,Scared to disclose So it’s been a couple month...,7,4,2022-03-19 19:55:19-04:00,scared to disclose so it’s been a couple month...,"[scared, to, disclose, so, it’s, been, a, coup..."
987,ti8tus,Hi. I just got diagnosed today and I am devast...,4,3,2022-03-19 20:21:23-04:00,hi i just got diagnosed today and i am devasta...,"[hi, i, just, got, diagnosed, today, and, i, a..."
988,tidj6p,First time disclosing Met someone on tinder an...,3,1,2022-03-20 00:52:55-04:00,first time disclosing met someone on tinder an...,"[first, time, disclosing, met, someone, on, ti..."


In [18]:
herpes1['tokenization'] = herpes1['tokenization'].apply(lambda x: [w for w in x if not w in 
                              stopwords.words('english')]) # Stem every word.
herpes1 # Print dataframe.

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text,tokenization
0,60xlxh,Be sure to read the sub RULES before asking ab...,24,56,2017-03-22 17:33:44-04:00,be sure to read the sub rules before asking ab...,"[sure, read, sub, rules, asking, personal, sym..."
1,npznfp,SPFPP 2021 HSV Survey Results Are in Here’s th...,142,172,2021-06-01 13:36:45-04:00,spfpp hsv survey results are in here’s the res...,"[spfpp, hsv, survey, results, here’s, results,..."
2,tynsw3,Hey! You are more than herpes! ✨ I don’t know ...,12,9,2022-04-07 17:50:44-04:00,hey you are more than herpes ✨ i don’t know if...,"[hey, herpes, ✨, don’t, know, post, allowed, s..."
3,typwuc,Day 1 I just left the Planned Parenthood where...,1,4,2022-04-07 19:33:30-04:00,day i just left the planned parenthood where t...,"[day, left, planned, parenthood, female, told,..."
4,typdhg,Western Blot Update I already knew I had hsv1 ...,4,6,2022-04-07 19:06:32-04:00,western blot update i already knew i had hsv b...,"[western, blot, update, already, knew, hsv, go..."
...,...,...,...,...,...,...,...
985,ticgwt,Can blisters go away without bursting?,8,2,2022-03-19 23:47:10-04:00,can blisters go away without bursting,"[blisters, go, away, without, bursting]"
986,ti8bec,Scared to disclose So it’s been a couple month...,7,4,2022-03-19 19:55:19-04:00,scared to disclose so it’s been a couple month...,"[scared, disclose, it’s, couple, months, since..."
987,ti8tus,Hi. I just got diagnosed today and I am devast...,4,3,2022-03-19 20:21:23-04:00,hi i just got diagnosed today and i am devasta...,"[hi, got, diagnosed, today, devastated, repost..."
988,tidj6p,First time disclosing Met someone on tinder an...,3,1,2022-03-20 00:52:55-04:00,first time disclosing met someone on tinder an...,"[first, time, disclosing, met, someone, tinder..."


In [19]:
ps = PorterStemmer()

In [20]:
#Stemming
herpes1['stemming'] = herpes1['tokenization'].apply(lambda x: [ps.stem(y) for y in x]) # Stem every word.
herpes1 # Print dataframe.

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text,tokenization,stemming
0,60xlxh,Be sure to read the sub RULES before asking ab...,24,56,2017-03-22 17:33:44-04:00,be sure to read the sub rules before asking ab...,"[sure, read, sub, rules, asking, personal, sym...","[sure, read, sub, rule, ask, person, symptom, ..."
1,npznfp,SPFPP 2021 HSV Survey Results Are in Here’s th...,142,172,2021-06-01 13:36:45-04:00,spfpp hsv survey results are in here’s the res...,"[spfpp, hsv, survey, results, here’s, results,...","[spfpp, hsv, survey, result, here’, result, su..."
2,tynsw3,Hey! You are more than herpes! ✨ I don’t know ...,12,9,2022-04-07 17:50:44-04:00,hey you are more than herpes ✨ i don’t know if...,"[hey, herpes, ✨, don’t, know, post, allowed, s...","[hey, herp, ✨, don’t, know, post, allow, sub, ..."
3,typwuc,Day 1 I just left the Planned Parenthood where...,1,4,2022-04-07 19:33:30-04:00,day i just left the planned parenthood where t...,"[day, left, planned, parenthood, female, told,...","[day, left, plan, parenthood, femal, told, tes..."
4,typdhg,Western Blot Update I already knew I had hsv1 ...,4,6,2022-04-07 19:06:32-04:00,western blot update i already knew i had hsv b...,"[western, blot, update, already, knew, hsv, go...","[western, blot, updat, alreadi, knew, hsv, got..."
...,...,...,...,...,...,...,...,...
985,ticgwt,Can blisters go away without bursting?,8,2,2022-03-19 23:47:10-04:00,can blisters go away without bursting,"[blisters, go, away, without, bursting]","[blister, go, away, without, burst]"
986,ti8bec,Scared to disclose So it’s been a couple month...,7,4,2022-03-19 19:55:19-04:00,scared to disclose so it’s been a couple month...,"[scared, disclose, it’s, couple, months, since...","[scare, disclos, it’, coupl, month, sinc, got,..."
987,ti8tus,Hi. I just got diagnosed today and I am devast...,4,3,2022-03-19 20:21:23-04:00,hi i just got diagnosed today and i am devasta...,"[hi, got, diagnosed, today, devastated, repost...","[hi, got, diagnos, today, devast, repost, thro..."
988,tidj6p,First time disclosing Met someone on tinder an...,3,1,2022-03-20 00:52:55-04:00,first time disclosing met someone on tinder an...,"[first, time, disclosing, met, someone, tinder...","[first, time, disclos, met, someon, tinder, we..."


In [21]:
lemmatizer = WordNetLemmatizer()

In [22]:
#Lemmatization
# a denotes adjective in "pos"
herpes1['lemmatizing'] = herpes1['stemming'].apply(lambda x: [lemmatizer.lemmatize(y, pos ="a") for y in x]) # Stem every word.
herpes1 # Print dataframe.

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text,tokenization,stemming,lemmatizing
0,60xlxh,Be sure to read the sub RULES before asking ab...,24,56,2017-03-22 17:33:44-04:00,be sure to read the sub rules before asking ab...,"[sure, read, sub, rules, asking, personal, sym...","[sure, read, sub, rule, ask, person, symptom, ...","[sure, read, sub, rule, ask, person, symptom, ..."
1,npznfp,SPFPP 2021 HSV Survey Results Are in Here’s th...,142,172,2021-06-01 13:36:45-04:00,spfpp hsv survey results are in here’s the res...,"[spfpp, hsv, survey, results, here’s, results,...","[spfpp, hsv, survey, result, here’, result, su...","[spfpp, hsv, survey, result, here’, result, su..."
2,tynsw3,Hey! You are more than herpes! ✨ I don’t know ...,12,9,2022-04-07 17:50:44-04:00,hey you are more than herpes ✨ i don’t know if...,"[hey, herpes, ✨, don’t, know, post, allowed, s...","[hey, herp, ✨, don’t, know, post, allow, sub, ...","[hey, herp, ✨, don’t, know, post, allow, sub, ..."
3,typwuc,Day 1 I just left the Planned Parenthood where...,1,4,2022-04-07 19:33:30-04:00,day i just left the planned parenthood where t...,"[day, left, planned, parenthood, female, told,...","[day, left, plan, parenthood, femal, told, tes...","[day, left, plan, parenthood, femal, told, tes..."
4,typdhg,Western Blot Update I already knew I had hsv1 ...,4,6,2022-04-07 19:06:32-04:00,western blot update i already knew i had hsv b...,"[western, blot, update, already, knew, hsv, go...","[western, blot, updat, alreadi, knew, hsv, got...","[western, blot, updat, alreadi, knew, hsv, got..."
...,...,...,...,...,...,...,...,...,...
985,ticgwt,Can blisters go away without bursting?,8,2,2022-03-19 23:47:10-04:00,can blisters go away without bursting,"[blisters, go, away, without, bursting]","[blister, go, away, without, burst]","[blister, go, away, without, burst]"
986,ti8bec,Scared to disclose So it’s been a couple month...,7,4,2022-03-19 19:55:19-04:00,scared to disclose so it’s been a couple month...,"[scared, disclose, it’s, couple, months, since...","[scare, disclos, it’, coupl, month, sinc, got,...","[scare, disclos, it’, coupl, month, sinc, got,..."
987,ti8tus,Hi. I just got diagnosed today and I am devast...,4,3,2022-03-19 20:21:23-04:00,hi i just got diagnosed today and i am devasta...,"[hi, got, diagnosed, today, devastated, repost...","[hi, got, diagnos, today, devast, repost, thro...","[hi, got, diagnos, today, devast, repost, thro..."
988,tidj6p,First time disclosing Met someone on tinder an...,3,1,2022-03-20 00:52:55-04:00,first time disclosing met someone on tinder an...,"[first, time, disclosing, met, someone, tinder...","[first, time, disclos, met, someon, tinder, we...","[first, time, disclos, met, someon, tinder, we..."


In [23]:
#combine tokenize words into sentance
herpes1["Text"] = [' '.join(t) for t in herpes1["lemmatizing"]]
herpes1

,ID,Post_Text,Total_Comments,Score,Post_Created,clean_text,tokenization,stemming,lemmatizing,Text
0,60xlxh,Be sure to read the sub RULES before asking ab...,24,56,2017-03-22 17:33:44-04:00,be sure to read the sub rules before asking ab...,"[sure, read, sub, rules, asking, personal, sym...","[sure, read, sub, rule, ask, person, symptom, ...","[sure, read, sub, rule, ask, person, symptom, ...",sure read sub rule ask person symptom wish ask...
1,npznfp,SPFPP 2021 HSV Survey Results Are in Here’s th...,142,172,2021-06-01 13:36:45-04:00,spfpp hsv survey results are in here’s the res...,"[spfpp, hsv, survey, results, here’s, results,...","[spfpp, hsv, survey, result, here’, result, su...","[spfpp, hsv, survey, result, here’, result, su...",spfpp hsv survey result here’ result survey co...
2,tynsw3,Hey! You are more than herpes! ✨ I don’t know ...,12,9,2022-04-07 17:50:44-04:00,hey you are more than herpes ✨ i don’t know if...,"[hey, herpes, ✨, don’t, know, post, allowed, s...","[hey, herp, ✨, don’t, know, post, allow, sub, ...","[hey, herp, ✨, don’t, know, post, allow, sub, ...",hey herp ✨ don’t know post allow sub realli he...
3,typwuc,Day 1 I just left the Planned Parenthood where...,1,4,2022-04-07 19:33:30-04:00,day i just left the planned parenthood where t...,"[day, left, planned, parenthood, female, told,...","[day, left, plan, parenthood, femal, told, tes...","[day, left, plan, parenthood, femal, told, tes...",day left plan parenthood femal told test point...
4,typdhg,Western Blot Update I already knew I had hsv1 ...,4,6,2022-04-07 19:06:32-04:00,western blot update i already knew i had hsv b...,"[western, blot, update, already, knew, hsv, go...","[western, blot, updat, alreadi, knew, hsv, got...","[western, blot, updat, alreadi, knew, hsv, got...",western blot updat alreadi knew hsv got ex man...
...,...,...,...,...,...,...,...,...,...,...
985,ticgwt,Can blisters go away without bursting?,8,2,2022-03-19 23:47:10-04:00,can blisters go away without bursting,"[blisters, go, away, without, bursting]","[blister, go, away, without, burst]","[blister, go, away, without, burst]",blister go away without burst
986,ti8bec,Scared to disclose So it’s been a couple month...,7,4,2022-03-19 19:55:19-04:00,scared to disclose so it’s been a couple month...,"[scared, disclose, it’s, couple, months, since...","[scare, disclos, it’, coupl, month, sinc, got,...","[scare, disclos, it’, coupl, month, sinc, got,...",scare disclos it’ coupl month sinc got diagnos...
987,ti8tus,Hi. I just got diagnosed today and I am devast...,4,3,2022-03-19 20:21:23-04:00,hi i just got diagnosed today and i am devasta...,"[hi, got, diagnosed, today, devastated, repost...","[hi, got, diagnos, today, devast, repost, thro...","[hi, got, diagnos, today, devast, repost, thro...",hi got diagnos today devast repost throwaway a...
988,tidj6p,First time disclosing Met someone on tinder an...,3,1,2022-03-20 00:52:55-04:00,first time disclosing met someone on tinder an...,"[first, time, disclosing, met, someone, tinder...","[first, time, disclos, met, someon, tinder, we...","[first, time, disclos, met, someon, tinder, we...",first time disclos met someon tinder we’v talk...


In [24]:
df_herpes1 = herpes1[["ID", "Post_Text", "Total_Comments", "Score", "Post_Created", "Text"]]
df_herpes1

,ID,Post_Text,Total_Comments,Score,Post_Created,Text
0,60xlxh,Be sure to read the sub RULES before asking ab...,24,56,2017-03-22 17:33:44-04:00,sure read sub rule ask person symptom wish ask...
1,npznfp,SPFPP 2021 HSV Survey Results Are in Here’s th...,142,172,2021-06-01 13:36:45-04:00,spfpp hsv survey result here’ result survey co...
2,tynsw3,Hey! You are more than herpes! ✨ I don’t know ...,12,9,2022-04-07 17:50:44-04:00,hey herp ✨ don’t know post allow sub realli he...
3,typwuc,Day 1 I just left the Planned Parenthood where...,1,4,2022-04-07 19:33:30-04:00,day left plan parenthood femal told test point...
4,typdhg,Western Blot Update I already knew I had hsv1 ...,4,6,2022-04-07 19:06:32-04:00,western blot updat alreadi knew hsv got ex man...
...,...,...,...,...,...,...
985,ticgwt,Can blisters go away without bursting?,8,2,2022-03-19 23:47:10-04:00,blister go away without burst
986,ti8bec,Scared to disclose So it’s been a couple month...,7,4,2022-03-19 19:55:19-04:00,scare disclos it’ coupl month sinc got diagnos...
987,ti8tus,Hi. I just got diagnosed today and I am devast...,4,3,2022-03-19 20:21:23-04:00,hi got diagnos today devast repost throwaway a...
988,tidj6p,First time disclosing Met someone on tinder an...,3,1,2022-03-20 00:52:55-04:00,first time disclos met someon tinder we’v talk...


In [25]:
#Getting sentiment scores
from textblob import TextBlob

In [26]:
## Define a function which can be applied to calculate the score for the whole dataset
 
def senti(x):
    return TextBlob(x).sentiment  
 
df_herpes1['senti_score'] = df_herpes1['Text'].apply(senti)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
df_herpes1
#As it can be observed there are two scores: the first score is sentiment polarity which tells if the sentiment is positive or negative 
  #and the second score is subjectivity score to tell how subjective is the text.
#The score indicates how negative or positive the overall text analyzed is. Anything below a score of -0.05 we tag as negative 
  #and anything above 0.05 we tag as positive. Anything in between inclusively, we tag as neutral.

,ID,Post_Text,Total_Comments,Score,Post_Created,Text,senti_score
0,60xlxh,Be sure to read the sub RULES before asking ab...,24,56,2017-03-22 17:33:44-04:00,sure read sub rule ask person symptom wish ask...,"(0.3666666666666667, 0.7259259259259259)"
1,npznfp,SPFPP 2021 HSV Survey Results Are in Here’s th...,142,172,2021-06-01 13:36:45-04:00,spfpp hsv survey result here’ result survey co...,"(0.2, 0.2)"
2,tynsw3,Hey! You are more than herpes! ✨ I don’t know ...,12,9,2022-04-07 17:50:44-04:00,hey herp ✨ don’t know post allow sub realli he...,"(0.4066666666666666, 0.5466666666666666)"
3,typwuc,Day 1 I just left the Planned Parenthood where...,1,4,2022-04-07 19:33:30-04:00,day left plan parenthood femal told test point...,"(-0.1037433155080214, 0.39117647058823524)"
4,typdhg,Western Blot Update I already knew I had hsv1 ...,4,6,2022-04-07 19:06:32-04:00,western blot updat alreadi knew hsv got ex man...,"(-0.08607954545454546, 0.37244318181818187)"
...,...,...,...,...,...,...,...
985,ticgwt,Can blisters go away without bursting?,8,2,2022-03-19 23:47:10-04:00,blister go away without burst,"(0.0, 0.0)"
986,ti8bec,Scared to disclose So it’s been a couple month...,7,4,2022-03-19 19:55:19-04:00,scare disclos it’ coupl month sinc got diagnos...,"(0.10335497835497835, 0.46396103896103896)"
987,ti8tus,Hi. I just got diagnosed today and I am devast...,4,3,2022-03-19 20:21:23-04:00,hi got diagnos today devast repost throwaway a...,"(0.09111111111111111, 0.4974074074074074)"
988,tidj6p,First time disclosing Met someone on tinder an...,3,1,2022-03-20 00:52:55-04:00,first time disclos met someon tinder we’v talk...,"(0.027272727272727258, 0.36212121212121207)"


In [28]:
df_herpes1["senti_score"][1]

Sentiment(polarity=0.2, subjectivity=0.2)

In [29]:
df_herpes1["polarity"] = [t[0] for t in df_herpes1["senti_score"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
df_herpes1["polarity"][1]

0.2

In [31]:
df_herpes1["type"] = ["negative" if t <= -0.05 else "positive" if t >= 0.05 else "nuetral" for t in df_herpes1["polarity"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
df_herpes1

,ID,Post_Text,Total_Comments,Score,Post_Created,Text,senti_score,polarity,type
0,60xlxh,Be sure to read the sub RULES before asking ab...,24,56,2017-03-22 17:33:44-04:00,sure read sub rule ask person symptom wish ask...,"(0.3666666666666667, 0.7259259259259259)",0.366667,positive
1,npznfp,SPFPP 2021 HSV Survey Results Are in Here’s th...,142,172,2021-06-01 13:36:45-04:00,spfpp hsv survey result here’ result survey co...,"(0.2, 0.2)",0.200000,positive
2,tynsw3,Hey! You are more than herpes! ✨ I don’t know ...,12,9,2022-04-07 17:50:44-04:00,hey herp ✨ don’t know post allow sub realli he...,"(0.4066666666666666, 0.5466666666666666)",0.406667,positive
3,typwuc,Day 1 I just left the Planned Parenthood where...,1,4,2022-04-07 19:33:30-04:00,day left plan parenthood femal told test point...,"(-0.1037433155080214, 0.39117647058823524)",-0.103743,negative
4,typdhg,Western Blot Update I already knew I had hsv1 ...,4,6,2022-04-07 19:06:32-04:00,western blot updat alreadi knew hsv got ex man...,"(-0.08607954545454546, 0.37244318181818187)",-0.086080,negative
...,...,...,...,...,...,...,...,...,...
985,ticgwt,Can blisters go away without bursting?,8,2,2022-03-19 23:47:10-04:00,blister go away without burst,"(0.0, 0.0)",0.000000,nuetral
986,ti8bec,Scared to disclose So it’s been a couple month...,7,4,2022-03-19 19:55:19-04:00,scare disclos it’ coupl month sinc got diagnos...,"(0.10335497835497835, 0.46396103896103896)",0.103355,positive
987,ti8tus,Hi. I just got diagnosed today and I am devast...,4,3,2022-03-19 20:21:23-04:00,hi got diagnos today devast repost throwaway a...,"(0.09111111111111111, 0.4974074074074074)",0.091111,positive
988,tidj6p,First time disclosing Met someone on tinder an...,3,1,2022-03-20 00:52:55-04:00,first time disclos met someon tinder we’v talk...,"(0.027272727272727258, 0.36212121212121207)",0.027273,nuetral


In [33]:
df_herpes1.to_csv('HERPES_POST.csv',index=False)

#Code for Comment_Text

In [34]:
herpes2 = pd.read_csv("https://raw.githubusercontent.com/SrushtiChauhan/Test-2022/main/HERPES_comments_individual.csv", 
                             on_bad_lines='skip');

In [35]:
print('HERPES Comments data:', herpes2.shape)


HERPES Comments data: (8000, 4)


In [36]:
herpes2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               8000 non-null   object
 1   Comment          8000 non-null   object
 2   Comment_Created  8000 non-null   object
 3   Score            8000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 250.1+ KB


In [37]:
herpes2.describe()

,Score
count,8000.000000
mean,2.089125
std,3.107513
min,-14.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,59.000000


In [38]:
herpes2.isnull().sum()

ID                 0
Comment            0
Comment_Created    0
Score              0
dtype: int64

In [39]:
herpes2["Comment"] = herpes2["Comment"].str.replace("[deleted]","");
herpes2["Comment"] = herpes2["Comment"].str.replace("[removed]","");

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [40]:
herpes2.isnull().sum()

ID                 0
Comment            0
Comment_Created    0
Score              0
dtype: int64

In [41]:
# Selecting duplicate rows except first
# occurrence based on all columns
duplicate = herpes2[herpes2.duplicated()]
 
print("Duplicate Rows :")
 
# Print the resultant Dataframe
duplicate

Duplicate Rows :


,ID,Comment,Comment_Created,Score


In [42]:
#Data Pre-Processing
herpes2.columns

Index(['ID', 'Comment', 'Comment_Created', 'Score'], dtype='object')

In [43]:
herpes2.head(3)

,ID,Comment,Comment_Created,Score
0,60xlxh,[],2017-07-20 15:30:34,1
1,60xlxh,Hi h. I' wning if w a si aw ps phs? I n' s hi...,2022-03-23 07:02:58,1
2,60xlxh,[],2017-07-20 16:33:17,1


In [44]:
#Convert comments data into lower case
herpes2['clean_text'] = herpes2["Comment"].str.lower()
herpes2

,ID,Comment,Comment_Created,Score,clean_text
0,60xlxh,[],2017-07-20 15:30:34,1,[]
1,60xlxh,Hi h. I' wning if w a si aw ps phs? I n' s hi...,2022-03-23 07:02:58,1,hi h. i' wning if w a si aw ps phs? i n' s hi...
2,60xlxh,[],2017-07-20 16:33:17,1,[]
3,60xlxh,T ps phs in h sub yu n fis upa h an iag hsin...,2022-03-24 00:01:17,2,t ps phs in h sub yu n fis upa h an iag hsin...
4,60xlxh,Thank yu s uch,2022-03-26 23:40:04,2,thank yu s uch
...,...,...,...,...,...
7995,ti86gw,Bn in h sa siuain an i a f b nc I sai shing ...,2022-03-20 01:33:10,1,bn in h sa siuain an i a f b nc i sai shing ...
7996,ti86gw,An n i’s n a f su hing sh g i.. y bf gs h a h ...,2022-03-20 01:33:48,1,an n i’s n a f su hing sh g i.. y bf gs h a h ...
7997,ti86gw,I can' i an say I n' g ging wih h psn ha ga i ...,2022-03-20 06:08:02,1,i can' i an say i n' g ging wih h psn ha ga i ...
7998,ti86gw,Uniky bh. Yu shu h hugh. \n\nYu can as h in w...,2022-03-20 06:43:41,1,uniky bh. yu shu h hugh. \n\nyu can as h in w...


In [45]:
#Autocorrect and spelling mistake correction
#spell = Speller(lang='en')

#herpes2["clean_text"] = [spell(t) for t in herpes2["clean_text"]]
#herpes2

In [46]:
#Remove punctuation and special characters from text data
herpes2["clean_text"] = [clean_text(t) for t in herpes2['clean_text']]
herpes2

,ID,Comment,Comment_Created,Score,clean_text
0,60xlxh,[],2017-07-20 15:30:34,1,
1,60xlxh,Hi h. I' wning if w a si aw ps phs? I n' s hi...,2022-03-23 07:02:58,1,hi h i wning if w a si aw ps phs i n s his pin...
2,60xlxh,[],2017-07-20 16:33:17,1,
3,60xlxh,T ps phs in h sub yu n fis upa h an iag hsin...,2022-03-24 00:01:17,2,t ps phs in h sub yu n fis upa h an iag hsing ...
4,60xlxh,Thank yu s uch,2022-03-26 23:40:04,2,thank yu s uch
...,...,...,...,...,...
7995,ti86gw,Bn in h sa siuain an i a f b nc I sai shing ...,2022-03-20 01:33:10,1,bn in h sa siuain an i a f b nc i sai shing h ...
7996,ti86gw,An n i’s n a f su hing sh g i.. y bf gs h a h ...,2022-03-20 01:33:48,1,an n i’s n a f su hing sh g i y bf gs h a h i ...
7997,ti86gw,I can' i an say I n' g ging wih h psn ha ga i ...,2022-03-20 06:08:02,1,i can i an say i n g ging wih h psn ha ga i bu...
7998,ti86gw,Uniky bh. Yu shu h hugh. \n\nYu can as h in w...,2022-03-20 06:43:41,1,uniky bh yu shu h hugh yu can as h in wih cain...


In [47]:
herpes2 = herpes2[herpes2["clean_text"].str.len() != 0]

In [48]:
#Tokenization
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()

herpes2["tokenization"] = [tk.tokenize(t) for t in herpes2["clean_text"]]
herpes2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,ID,Comment,Comment_Created,Score,clean_text,tokenization
1,60xlxh,Hi h. I' wning if w a si aw ps phs? I n' s hi...,2022-03-23 07:02:58,1,hi h i wning if w a si aw ps phs i n s his pin...,"[hi, h, i, wning, if, w, a, si, aw, ps, phs, i..."
3,60xlxh,T ps phs in h sub yu n fis upa h an iag hsin...,2022-03-24 00:01:17,2,t ps phs in h sub yu n fis upa h an iag hsing ...,"[t, ps, phs, in, h, sub, yu, n, fis, upa, h, a..."
4,60xlxh,Thank yu s uch,2022-03-26 23:40:04,2,thank yu s uch,"[thank, yu, s, uch]"
5,npznfp,Insing knw h ajiy has succssfu iscsus an ha s...,2021-06-02 03:16:36,23,insing knw h ajiy has succssfu iscsus an ha sx...,"[insing, knw, h, ajiy, has, succssfu, iscsus, ..."
6,npznfp,I aing his. Thank yu f shaing. I' paicipa i...,2021-06-13 04:17:11,12,i aing his thank yu f shaing i paicipa in any ...,"[i, aing, his, thank, yu, f, shaing, i, paicip..."
...,...,...,...,...,...,...
7995,ti86gw,Bn in h sa siuain an i a f b nc I sai shing ...,2022-03-20 01:33:10,1,bn in h sa siuain an i a f b nc i sai shing h ...,"[bn, in, h, sa, siuain, an, i, a, f, b, nc, i,..."
7996,ti86gw,An n i’s n a f su hing sh g i.. y bf gs h a h ...,2022-03-20 01:33:48,1,an n i’s n a f su hing sh g i y bf gs h a h i ...,"[an, n, i’s, n, a, f, su, hing, sh, g, i, y, b..."
7997,ti86gw,I can' i an say I n' g ging wih h psn ha ga i ...,2022-03-20 06:08:02,1,i can i an say i n g ging wih h psn ha ga i bu...,"[i, can, i, an, say, i, n, g, ging, wih, h, ps..."
7998,ti86gw,Uniky bh. Yu shu h hugh. \n\nYu can as h in w...,2022-03-20 06:43:41,1,uniky bh yu shu h hugh yu can as h in wih cain...,"[uniky, bh, yu, shu, h, hugh, yu, can, as, h, ..."


In [49]:
herpes2['tokenization'] = herpes2['tokenization'].apply(lambda x: [w for w in x if not w in 
                              stopwords.words('english')]) # Stem every word.
herpes2 # Print dataframe.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Comment,Comment_Created,Score,clean_text,tokenization
1,60xlxh,Hi h. I' wning if w a si aw ps phs? I n' s hi...,2022-03-23 07:02:58,1,hi h i wning if w a si aw ps phs i n s his pin...,"[hi, h, wning, w, si, aw, ps, phs, n, pin, whn..."
3,60xlxh,T ps phs in h sub yu n fis upa h an iag hsin...,2022-03-24 00:01:17,2,t ps phs in h sub yu n fis upa h an iag hsing ...,"[ps, phs, h, sub, yu, n, fis, upa, h, iag, hsi..."
4,60xlxh,Thank yu s uch,2022-03-26 23:40:04,2,thank yu s uch,"[thank, yu, uch]"
5,npznfp,Insing knw h ajiy has succssfu iscsus an ha s...,2021-06-02 03:16:36,23,insing knw h ajiy has succssfu iscsus an ha sx...,"[insing, knw, h, ajiy, succssfu, iscsus, ha, s..."
6,npznfp,I aing his. Thank yu f shaing. I' paicipa i...,2021-06-13 04:17:11,12,i aing his thank yu f shaing i paicipa in any ...,"[aing, thank, yu, f, shaing, paicipa, fuu, suys]"
...,...,...,...,...,...,...
7995,ti86gw,Bn in h sa siuain an i a f b nc I sai shing ...,2022-03-20 01:33:10,1,bn in h sa siuain an i a f b nc i sai shing h ...,"[bn, h, sa, siuain, f, b, nc, sai, shing, h, p..."
7996,ti86gw,An n i’s n a f su hing sh g i.. y bf gs h a h ...,2022-03-20 01:33:48,1,an n i’s n a f su hing sh g i y bf gs h a h i ...,"[n, i’s, n, f, su, hing, sh, g, bf, gs, h, h, ..."
7997,ti86gw,I can' i an say I n' g ging wih h psn ha ga i ...,2022-03-20 06:08:02,1,i can i an say i n g ging wih h psn ha ga i bu...,"[say, n, g, ging, wih, h, psn, ha, ga, bu, f, ..."
7998,ti86gw,Uniky bh. Yu shu h hugh. \n\nYu can as h in w...,2022-03-20 06:43:41,1,uniky bh yu shu h hugh yu can as h in wih cain...,"[uniky, bh, yu, shu, h, hugh, yu, h, wih, cain..."


In [50]:
ps = PorterStemmer()

In [51]:
#Stemming
herpes2['stemming'] = herpes2['tokenization'].apply(lambda x: [ps.stem(y) for y in x]) # Stem every word.
herpes2 # Print dataframe.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,Comment,Comment_Created,Score,clean_text,tokenization,stemming
1,60xlxh,Hi h. I' wning if w a si aw ps phs? I n' s hi...,2022-03-23 07:02:58,1,hi h i wning if w a si aw ps phs i n s his pin...,"[hi, h, wning, w, si, aw, ps, phs, n, pin, whn...","[hi, h, wning, w, si, aw, ps, ph, n, pin, whn,..."
3,60xlxh,T ps phs in h sub yu n fis upa h an iag hsin...,2022-03-24 00:01:17,2,t ps phs in h sub yu n fis upa h an iag hsing ...,"[ps, phs, h, sub, yu, n, fis, upa, h, iag, hsi...","[ps, ph, h, sub, yu, n, fi, upa, h, iag, hsing..."
4,60xlxh,Thank yu s uch,2022-03-26 23:40:04,2,thank yu s uch,"[thank, yu, uch]","[thank, yu, uch]"
5,npznfp,Insing knw h ajiy has succssfu iscsus an ha s...,2021-06-02 03:16:36,23,insing knw h ajiy has succssfu iscsus an ha sx...,"[insing, knw, h, ajiy, succssfu, iscsus, ha, s...","[ins, knw, h, ajiy, succssfu, iscsu, ha, sx, w..."
6,npznfp,I aing his. Thank yu f shaing. I' paicipa i...,2021-06-13 04:17:11,12,i aing his thank yu f shaing i paicipa in any ...,"[aing, thank, yu, f, shaing, paicipa, fuu, suys]","[a, thank, yu, f, sha, paicipa, fuu, suy]"
...,...,...,...,...,...,...,...
7995,ti86gw,Bn in h sa siuain an i a f b nc I sai shing ...,2022-03-20 01:33:10,1,bn in h sa siuain an i a f b nc i sai shing h ...,"[bn, h, sa, siuain, f, b, nc, sai, shing, h, p...","[bn, h, sa, siuain, f, b, nc, sai, shing, h, p..."
7996,ti86gw,An n i’s n a f su hing sh g i.. y bf gs h a h ...,2022-03-20 01:33:48,1,an n i’s n a f su hing sh g i y bf gs h a h i ...,"[n, i’s, n, f, su, hing, sh, g, bf, gs, h, h, ...","[n, i’, n, f, su, hing, sh, g, bf, gs, h, h, i..."
7997,ti86gw,I can' i an say I n' g ging wih h psn ha ga i ...,2022-03-20 06:08:02,1,i can i an say i n g ging wih h psn ha ga i bu...,"[say, n, g, ging, wih, h, psn, ha, ga, bu, f, ...","[say, n, g, ging, wih, h, psn, ha, ga, bu, f, ..."
7998,ti86gw,Uniky bh. Yu shu h hugh. \n\nYu can as h in w...,2022-03-20 06:43:41,1,uniky bh yu shu h hugh yu can as h in wih cain...,"[uniky, bh, yu, shu, h, hugh, yu, h, wih, cain...","[uniki, bh, yu, shu, h, hugh, yu, h, wih, ca, ..."


In [52]:
lemmatizer = WordNetLemmatizer()

In [53]:
#Lemmatization
# a denotes adjective in "pos"
herpes2['lemmatizing'] = herpes2['stemming'].apply(lambda x: [lemmatizer.lemmatize(y, pos ="a") for y in x]) # Stem every word.
herpes2 # Print dataframe.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ID,Comment,Comment_Created,Score,clean_text,tokenization,stemming,lemmatizing
1,60xlxh,Hi h. I' wning if w a si aw ps phs? I n' s hi...,2022-03-23 07:02:58,1,hi h i wning if w a si aw ps phs i n s his pin...,"[hi, h, wning, w, si, aw, ps, phs, n, pin, whn...","[hi, h, wning, w, si, aw, ps, ph, n, pin, whn,...","[hi, h, wning, w, si, aw, ps, ph, n, pin, whn,..."
3,60xlxh,T ps phs in h sub yu n fis upa h an iag hsin...,2022-03-24 00:01:17,2,t ps phs in h sub yu n fis upa h an iag hsing ...,"[ps, phs, h, sub, yu, n, fis, upa, h, iag, hsi...","[ps, ph, h, sub, yu, n, fi, upa, h, iag, hsing...","[ps, ph, h, sub, yu, n, fi, upa, h, iag, hsing..."
4,60xlxh,Thank yu s uch,2022-03-26 23:40:04,2,thank yu s uch,"[thank, yu, uch]","[thank, yu, uch]","[thank, yu, uch]"
5,npznfp,Insing knw h ajiy has succssfu iscsus an ha s...,2021-06-02 03:16:36,23,insing knw h ajiy has succssfu iscsus an ha sx...,"[insing, knw, h, ajiy, succssfu, iscsus, ha, s...","[ins, knw, h, ajiy, succssfu, iscsu, ha, sx, w...","[ins, knw, h, ajiy, succssfu, iscsu, ha, sx, w..."
6,npznfp,I aing his. Thank yu f shaing. I' paicipa i...,2021-06-13 04:17:11,12,i aing his thank yu f shaing i paicipa in any ...,"[aing, thank, yu, f, shaing, paicipa, fuu, suys]","[a, thank, yu, f, sha, paicipa, fuu, suy]","[a, thank, yu, f, sha, paicipa, fuu, suy]"
...,...,...,...,...,...,...,...,...
7995,ti86gw,Bn in h sa siuain an i a f b nc I sai shing ...,2022-03-20 01:33:10,1,bn in h sa siuain an i a f b nc i sai shing h ...,"[bn, h, sa, siuain, f, b, nc, sai, shing, h, p...","[bn, h, sa, siuain, f, b, nc, sai, shing, h, p...","[bn, h, sa, siuain, f, b, nc, sai, shing, h, p..."
7996,ti86gw,An n i’s n a f su hing sh g i.. y bf gs h a h ...,2022-03-20 01:33:48,1,an n i’s n a f su hing sh g i y bf gs h a h i ...,"[n, i’s, n, f, su, hing, sh, g, bf, gs, h, h, ...","[n, i’, n, f, su, hing, sh, g, bf, gs, h, h, i...","[n, i’, n, f, su, hing, sh, g, bf, gs, h, h, i..."
7997,ti86gw,I can' i an say I n' g ging wih h psn ha ga i ...,2022-03-20 06:08:02,1,i can i an say i n g ging wih h psn ha ga i bu...,"[say, n, g, ging, wih, h, psn, ha, ga, bu, f, ...","[say, n, g, ging, wih, h, psn, ha, ga, bu, f, ...","[say, n, g, ging, wih, h, psn, ha, ga, bu, f, ..."
7998,ti86gw,Uniky bh. Yu shu h hugh. \n\nYu can as h in w...,2022-03-20 06:43:41,1,uniky bh yu shu h hugh yu can as h in wih cain...,"[uniky, bh, yu, shu, h, hugh, yu, h, wih, cain...","[uniki, bh, yu, shu, h, hugh, yu, h, wih, ca, ...","[uniki, bh, yu, shu, h, hugh, yu, h, wih, ca, ..."


In [54]:
#combine tokenize words into sentance
herpes2["Text"] = [' '.join(t) for t in herpes2["lemmatizing"]]
herpes2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,Comment,Comment_Created,Score,clean_text,tokenization,stemming,lemmatizing,Text
1,60xlxh,Hi h. I' wning if w a si aw ps phs? I n' s hi...,2022-03-23 07:02:58,1,hi h i wning if w a si aw ps phs i n s his pin...,"[hi, h, wning, w, si, aw, ps, phs, n, pin, whn...","[hi, h, wning, w, si, aw, ps, ph, n, pin, whn,...","[hi, h, wning, w, si, aw, ps, ph, n, pin, whn,...",hi h wning w si aw ps ph n pin whn ying ca ps
3,60xlxh,T ps phs in h sub yu n fis upa h an iag hsin...,2022-03-24 00:01:17,2,t ps phs in h sub yu n fis upa h an iag hsing ...,"[ps, phs, h, sub, yu, n, fis, upa, h, iag, hsi...","[ps, ph, h, sub, yu, n, fi, upa, h, iag, hsing...","[ps, ph, h, sub, yu, n, fi, upa, h, iag, hsing...",ps ph h sub yu n fi upa h iag hsing sic iguc h...
4,60xlxh,Thank yu s uch,2022-03-26 23:40:04,2,thank yu s uch,"[thank, yu, uch]","[thank, yu, uch]","[thank, yu, uch]",thank yu uch
5,npznfp,Insing knw h ajiy has succssfu iscsus an ha s...,2021-06-02 03:16:36,23,insing knw h ajiy has succssfu iscsus an ha sx...,"[insing, knw, h, ajiy, succssfu, iscsus, ha, s...","[ins, knw, h, ajiy, succssfu, iscsu, ha, sx, w...","[ins, knw, h, ajiy, succssfu, iscsu, ha, sx, w...",ins knw h ajiy succssfu iscsu ha sx wih nn psi...
6,npznfp,I aing his. Thank yu f shaing. I' paicipa i...,2021-06-13 04:17:11,12,i aing his thank yu f shaing i paicipa in any ...,"[aing, thank, yu, f, shaing, paicipa, fuu, suys]","[a, thank, yu, f, sha, paicipa, fuu, suy]","[a, thank, yu, f, sha, paicipa, fuu, suy]",a thank yu f sha paicipa fuu suy
...,...,...,...,...,...,...,...,...,...
7995,ti86gw,Bn in h sa siuain an i a f b nc I sai shing ...,2022-03-20 01:33:10,1,bn in h sa siuain an i a f b nc i sai shing h ...,"[bn, h, sa, siuain, f, b, nc, sai, shing, h, p...","[bn, h, sa, siuain, f, b, nc, sai, shing, h, p...","[bn, h, sa, siuain, f, b, nc, sai, shing, h, p...",bn h sa siuain f b nc sai shing h psn yu gui g...
7996,ti86gw,An n i’s n a f su hing sh g i.. y bf gs h a h ...,2022-03-20 01:33:48,1,an n i’s n a f su hing sh g i y bf gs h a h i ...,"[n, i’s, n, f, su, hing, sh, g, bf, gs, h, h, ...","[n, i’, n, f, su, hing, sh, g, bf, gs, h, h, i...","[n, i’, n, f, su, hing, sh, g, bf, gs, h, h, i...",n i’ n f su hing sh g bf gs h h i’ fin
7997,ti86gw,I can' i an say I n' g ging wih h psn ha ga i ...,2022-03-20 06:08:02,1,i can i an say i n g ging wih h psn ha ga i bu...,"[say, n, g, ging, wih, h, psn, ha, ga, bu, f, ...","[say, n, g, ging, wih, h, psn, ha, ga, bu, f, ...","[say, n, g, ging, wih, h, psn, ha, ga, bu, f, ...",say n g ging wih h psn ha ga bu f ha bcau n ik...
7998,ti86gw,Uniky bh. Yu shu h hugh. \n\nYu can as h in w...,2022-03-20 06:43:41,1,uniky bh yu shu h hugh yu can as h in wih cain...,"[uniky, bh, yu, shu, h, hugh, yu, h, wih, cain...","[uniki, bh, yu, shu, h, hugh, yu, h, wih, ca, ...","[uniki, bh, yu, shu, h, hugh, yu, h, wih, ca, ...",uniki bh yu shu h hugh yu h wih ca c sinc assu...


In [55]:
herpes2.isnull().sum()

ID                 0
Comment            0
Comment_Created    0
Score              0
clean_text         0
tokenization       0
stemming           0
lemmatizing        0
Text               0
dtype: int64

In [56]:
herpes2.columns

Index(['ID', 'Comment', 'Comment_Created', 'Score', 'clean_text',
       'tokenization', 'stemming', 'lemmatizing', 'Text'],
      dtype='object')

In [57]:
df_herpes2 = herpes2[["ID", "Comment", "Comment_Created", "Score", "Text"]]
df_herpes2

,ID,Comment,Comment_Created,Score,Text
1,60xlxh,Hi h. I' wning if w a si aw ps phs? I n' s hi...,2022-03-23 07:02:58,1,hi h wning w si aw ps ph n pin whn ying ca ps
3,60xlxh,T ps phs in h sub yu n fis upa h an iag hsin...,2022-03-24 00:01:17,2,ps ph h sub yu n fi upa h iag hsing sic iguc h...
4,60xlxh,Thank yu s uch,2022-03-26 23:40:04,2,thank yu uch
5,npznfp,Insing knw h ajiy has succssfu iscsus an ha s...,2021-06-02 03:16:36,23,ins knw h ajiy succssfu iscsu ha sx wih nn psi...
6,npznfp,I aing his. Thank yu f shaing. I' paicipa i...,2021-06-13 04:17:11,12,a thank yu f sha paicipa fuu suy
...,...,...,...,...,...
7995,ti86gw,Bn in h sa siuain an i a f b nc I sai shing ...,2022-03-20 01:33:10,1,bn h sa siuain f b nc sai shing h psn yu gui g...
7996,ti86gw,An n i’s n a f su hing sh g i.. y bf gs h a h ...,2022-03-20 01:33:48,1,n i’ n f su hing sh g bf gs h h i’ fin
7997,ti86gw,I can' i an say I n' g ging wih h psn ha ga i ...,2022-03-20 06:08:02,1,say n g ging wih h psn ha ga bu f ha bcau n ik...
7998,ti86gw,Uniky bh. Yu shu h hugh. \n\nYu can as h in w...,2022-03-20 06:43:41,1,uniki bh yu shu h hugh yu h wih ca c sinc assu...


In [58]:
#Getting sentiment scores
from textblob import TextBlob

In [59]:
## Define a function which can be applied to calculate the score for the whole dataset
 
def senti(x):
    return TextBlob(x).sentiment  
 
df_herpes2['senti_score'] = df_herpes2['Text'].apply(senti)
df_herpes2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,Comment,Comment_Created,Score,Text,senti_score
1,60xlxh,Hi h. I' wning if w a si aw ps phs? I n' s hi...,2022-03-23 07:02:58,1,hi h wning w si aw ps ph n pin whn ying ca ps,"(0.0, 0.0)"
3,60xlxh,T ps phs in h sub yu n fis upa h an iag hsin...,2022-03-24 00:01:17,2,ps ph h sub yu n fi upa h iag hsing sic iguc h...,"(0.0, 0.0)"
4,60xlxh,Thank yu s uch,2022-03-26 23:40:04,2,thank yu uch,"(0.0, 0.0)"
5,npznfp,Insing knw h ajiy has succssfu iscsus an ha s...,2021-06-02 03:16:36,23,ins knw h ajiy succssfu iscsu ha sx wih nn psi...,"(0.0, 0.0)"
6,npznfp,I aing his. Thank yu f shaing. I' paicipa i...,2021-06-13 04:17:11,12,a thank yu f sha paicipa fuu suy,"(0.0, 0.0)"
...,...,...,...,...,...,...
7995,ti86gw,Bn in h sa siuain an i a f b nc I sai shing ...,2022-03-20 01:33:10,1,bn h sa siuain f b nc sai shing h psn yu gui g...,"(0.0, 0.0)"
7996,ti86gw,An n i’s n a f su hing sh g i.. y bf gs h a h ...,2022-03-20 01:33:48,1,n i’ n f su hing sh g bf gs h h i’ fin,"(0.0, 0.0)"
7997,ti86gw,I can' i an say I n' g ging wih h psn ha ga i ...,2022-03-20 06:08:02,1,say n g ging wih h psn ha ga bu f ha bcau n ik...,"(0.0, 0.0)"
7998,ti86gw,Uniky bh. Yu shu h hugh. \n\nYu can as h in w...,2022-03-20 06:43:41,1,uniki bh yu shu h hugh yu h wih ca c sinc assu...,"(-0.2, 0.15000000000000002)"


In [60]:
df_herpes2["polarity"] = [t[0] for t in df_herpes2["senti_score"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
df_herpes2["type"] = ["negative" if t <= -0.05 else "positive" if t >= 0.05 else "nuetral" for t in df_herpes2["polarity"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [62]:
df_herpes2

,ID,Comment,Comment_Created,Score,Text,senti_score,polarity,type
1,60xlxh,Hi h. I' wning if w a si aw ps phs? I n' s hi...,2022-03-23 07:02:58,1,hi h wning w si aw ps ph n pin whn ying ca ps,"(0.0, 0.0)",0.0,nuetral
3,60xlxh,T ps phs in h sub yu n fis upa h an iag hsin...,2022-03-24 00:01:17,2,ps ph h sub yu n fi upa h iag hsing sic iguc h...,"(0.0, 0.0)",0.0,nuetral
4,60xlxh,Thank yu s uch,2022-03-26 23:40:04,2,thank yu uch,"(0.0, 0.0)",0.0,nuetral
5,npznfp,Insing knw h ajiy has succssfu iscsus an ha s...,2021-06-02 03:16:36,23,ins knw h ajiy succssfu iscsu ha sx wih nn psi...,"(0.0, 0.0)",0.0,nuetral
6,npznfp,I aing his. Thank yu f shaing. I' paicipa i...,2021-06-13 04:17:11,12,a thank yu f sha paicipa fuu suy,"(0.0, 0.0)",0.0,nuetral
...,...,...,...,...,...,...,...,...
7995,ti86gw,Bn in h sa siuain an i a f b nc I sai shing ...,2022-03-20 01:33:10,1,bn h sa siuain f b nc sai shing h psn yu gui g...,"(0.0, 0.0)",0.0,nuetral
7996,ti86gw,An n i’s n a f su hing sh g i.. y bf gs h a h ...,2022-03-20 01:33:48,1,n i’ n f su hing sh g bf gs h h i’ fin,"(0.0, 0.0)",0.0,nuetral
7997,ti86gw,I can' i an say I n' g ging wih h psn ha ga i ...,2022-03-20 06:08:02,1,say n g ging wih h psn ha ga bu f ha bcau n ik...,"(0.0, 0.0)",0.0,nuetral
7998,ti86gw,Uniky bh. Yu shu h hugh. \n\nYu can as h in w...,2022-03-20 06:43:41,1,uniki bh yu shu h hugh yu h wih ca c sinc assu...,"(-0.2, 0.15000000000000002)",-0.2,negative


In [63]:
df_herpes2['type'].value_counts()

nuetral     6535
negative     796
positive     585
Name: type, dtype: int64

In [64]:
df_herpes2.to_csv('HERPES_COMMENT.csv',index=False)